In [53]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

%matplotlib inline

In [ ]:
classes = ('T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot')

In [54]:
train_transformations = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32,padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

test_transformations = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [55]:
trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, transform=train_transformations, download=True)
testset = torchvision.datasets.FashionMNIST(root='./data', train=False, transform=test_transformations, download=True)

In [56]:
#trainset, valset = train_test_split(trainset, test_size=0.2, random_state=42)

In [57]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=0)
#valloader = torch.utils.data.DataLoader(valset, batch_size=4, shuffle=False, num_workers=0)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=0)

In [85]:
class Net(nn.Module):
    def __init__(self, in_chan, out_chan):
        super(Net, self).__init__()
        self.conv = nn.Conv2d(in_channels=in_chan, out_channels=out_chan, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm2d(num_features=out_chan)
        self.drop = nn.Dropout(p=0.2)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = F.dropout(self.drop(x), training=self.training)
        x = self.relu(x)
        return x

class SimpleNet(nn.Module):
    def __init__(self, num_class=10):
        super(SimpleNet, self).__init__()
        self.net1 = Net(1,16)
        self.net2 = Net(16,32)
        self.net3 = Net(32,64)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.net4 = Net(64,64)
        self.net5 = Net(64,64)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.net6 = Net(64,128)
        self.net7 = Net(128,128)
        self.pool3 = nn.MaxPool2d(kernel_size=2)
        self.avgpool = nn.AvgPool2d(kernel_size=4) #change to kernal size 3?
        
        self.net = nn.Sequential(self.net1,self.net2,self.net3,self.pool1,self.net4,self.net5,self.pool2,self.net6,self.net7,self.pool3,self.avgpool)
        self.fc = nn.Linear(in_features=128,out_features=num_class)

    def forward(self, input):
        output = self.net(input)
        output = output.view(-1,128)
        output = self.fc(output)
        return output

In [86]:
model = SimpleNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [87]:
def adjust_learning_rate(epoch):
    lr = 0.001

    if epoch > 50:
        lr = lr / 1000000
    elif epoch > 40:
        lr = lr / 100000
    elif epoch > 30:
        lr = lr / 10000
    elif epoch > 20:
        lr = lr / 1000
    elif epoch > 10:
        lr = lr / 100
    elif epoch > 5:
        lr = lr / 10

    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

In [109]:
def train(num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            images, labels = data
            # Clear all accumulated gradients
            optimizer.zero_grad()
            # Predict classes using images from the test set
            outputs = model(images)
            # Compute the loss based on the predictions and actual labels
            loss = criterion(outputs, labels)
            # Backpropagate the loss
            loss.backward()
            # Adjust parameters according to the computed gradients
            optimizer.step()
            # Update running loss
            running_loss += loss.item()

            # Print the metrics
            if i % 2000 == 1999:
                print("Epoch {}, Count: {}, Running Loss: {}".format(epoch+1, i+1,running_loss))
        #adjust_learning_rate(epoch)

In [ ]:
train(5)

Epoch 1, Count: 2000, Running Loss: 5138.777373433113
Epoch 1, Count: 4000, Running Loss: 10292.068709373474
Epoch 1, Count: 6000, Running Loss: 15472.336253285408
Epoch 1, Count: 8000, Running Loss: 20628.961946964264
Epoch 1, Count: 10000, Running Loss: 25783.86703646183
Epoch 1, Count: 12000, Running Loss: 30919.46484029293
Epoch 1, Count: 14000, Running Loss: 36056.45467936993
Epoch 2, Count: 2000, Running Loss: 5164.9055869579315
Epoch 2, Count: 4000, Running Loss: 10314.213210701942
Epoch 2, Count: 6000, Running Loss: 15446.92931163311
Epoch 2, Count: 8000, Running Loss: 20582.429592370987
Epoch 2, Count: 10000, Running Loss: 25746.230667948723
Epoch 2, Count: 12000, Running Loss: 30910.976651906967
Epoch 2, Count: 14000, Running Loss: 36081.42679691315
Epoch 3, Count: 2000, Running Loss: 5132.892731428146
Epoch 3, Count: 4000, Running Loss: 10275.792593121529
Epoch 3, Count: 6000, Running Loss: 15437.320172786713
Epoch 3, Count: 8000, Running Loss: 20588.90808045864
Epoch 3, Cou

In [ ]:
PATH = './fashion-MNIST_e5.pth'
torch.save(model.state_dict(), PATH)

In [ ]:
net = SimpleNet()
net.load_state_dict(torch.load(PATH))

In [ ]:
def test():
    model.eval()
    test_acc = 0.0
    for i, (images, labels) in enumerate(test_loader):
        # Predict classes using images from the test set
        outputs = model(images)
        _, prediction = torch.max(outputs.data, 1)
        
        test_acc += torch.sum(prediction == labels.data)

    # Compute the average acc and loss over all 10000 test images
    test_acc = test_acc / 10000

    return test_acc